In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xf /content/drive/MyDrive/skin_cancer/final_data.tar -C /content/

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.losses import CategoricalFocalCrossentropy

# PATHS
TRAIN_DIR = "/content/final_data/train"
VAL_DIR = "/content/final_data/valid/sorted"
TEST_DIR = "/content/final_data/test/sorted"
LOCAL_CHECKPOINT_PATH = "/content/drive/MyDrive/skin_cancer/models/efficientnetv2_checkpoints"
FINAL_MODEL_PATH = "/content/drive/MyDrive/skin_cancer/models/efficientnetv2_full_model.h5"
os.makedirs(LOCAL_CHECKPOINT_PATH, exist_ok=True)
os.makedirs(os.path.dirname(FINAL_MODEL_PATH), exist_ok=True)

# PARAMETERS
IMG_SIZE = (300, 300)
BATCH_SIZE = 32
EPOCHS = 20

# CENTER CROP FUNCTION
def center_crop_and_preprocess(img):
    """Crop to center square, resize, then preprocess for EfficientNetV2."""
    h, w, _ = img.shape
    min_side = min(h, w)
    top = (h - min_side) // 2
    left = (w - min_side) // 2
    img = img[top:top + min_side, left:left + min_side]
    img = tf.image.resize(img, IMG_SIZE)
    img = preprocess_input(img)
    return img

# DATA AUGMENTATION
train_datagen = ImageDataGenerator(
    preprocessing_function=center_crop_and_preprocess,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    shear_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest",
)

val_datagen = ImageDataGenerator(preprocessing_function=center_crop_and_preprocess)
test_datagen = ImageDataGenerator(preprocessing_function=center_crop_and_preprocess)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=True
)
val_gen = val_datagen.flow_from_directory(
    VAL_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)
test_gen = test_datagen.flow_from_directory(
    TEST_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

# CLASS WEIGHTS → for Focal Loss alpha
labels = train_gen.classes
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights_dict = dict(zip(np.unique(labels), class_weights_array))
print("Class weights:", class_weights_dict)

# Normalize α for focal loss
alpha_array = np.array([class_weights_dict[i] for i in sorted(class_weights_dict.keys())], dtype=np.float32)
alpha_array = alpha_array / np.sum(alpha_array)
print("Alpha array:", alpha_array)

# MODEL (EfficientNetV2-S)
base_model = EfficientNetV2S(
    weights="imagenet",
    include_top=False,
    input_shape=(300, 300, 3)
)

# Freeze base model (warmup phase)
for layer in base_model.layers:
    layer.trainable = False

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dropout(0.4)(x)
x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
output = layers.Dense(train_gen.num_classes, activation="softmax")(x)

model = models.Model(inputs=base_model.input, outputs=output)

# LOSS FUNCTION
loss_fn = CategoricalFocalCrossentropy(
    gamma=2,
    alpha=alpha_array,
    label_smoothing=0.05,
    from_logits=False
)

# COMPILE (Warmup Phase)
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss=loss_fn,
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), AUC(name="auc")]
)

# CALLBACKS
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(LOCAL_CHECKPOINT_PATH, "ckpt-{epoch:02d}.keras"),
    save_weights_only=False,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)
early_stop = EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=4, min_lr=1e-6, verbose=1)

# PHASE 1: Warmup (Top Layers)
history_warmup = model.fit(
    train_gen,
    epochs=2,
    validation_data=val_gen,
    callbacks=[checkpoint_callback, early_stop, reduce_lr],
    verbose=1
)

# PHASE 2: Fine-tuning deeper layers
# Unfreeze top half of EfficientNetV2-S
fine_tune_at = len(base_model.layers) // 2
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in base_model.layers[fine_tune_at:]:
    layer.trainable = True

# Re-compile with smaller LR
model.compile(
    optimizer=optimizers.Adam(learning_rate=5e-5),
    loss=loss_fn,
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), AUC(name="auc")]
)

history_finetune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    initial_epoch=2,
    callbacks=[checkpoint_callback, early_stop, reduce_lr],
    verbose=1
)

# SAVE FINAL MODEL
model.save(FINAL_MODEL_PATH, save_format="tf")
print(f"✅ Training complete! Full model saved at {FINAL_MODEL_PATH}")

Found 27205 images belonging to 7 classes.
Found 235 images belonging to 7 classes.
Found 1470 images belonging to 7 classes.
Class weights: {np.int32(0): np.float64(1.2954761904761904), np.int32(1): np.float64(1.1104081632653062), np.int32(2): np.float64(0.9716071428571429), np.int32(3): np.float64(1.2954761904761904), np.int32(4): np.float64(0.9716071428571429), np.int32(5): np.float64(0.5796314051347609), np.int32(6): np.float64(1.2954761904761904)}
Alpha array: [0.17227803 0.14766689 0.12920852 0.17227803 0.12920852 0.07708189
 0.17227803]


KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.losses import CategoricalFocalCrossentropy

# PATHS
TRAIN_DIR = "/content/final_data/train"
VAL_DIR = "/content/final_data/valid/sorted"
TEST_DIR = "/content/final_data/test/sorted"
LOCAL_CHECKPOINT_PATH = "/content/drive/MyDrive/skin_cancer/models/efficientnetv2_checkpoints"
FINAL_MODEL_PATH = "/content/drive/MyDrive/skin_cancer/models/efficientnetv2_full_model.h5"
os.makedirs(LOCAL_CHECKPOINT_PATH, exist_ok=True)
os.makedirs(os.path.dirname(FINAL_MODEL_PATH), exist_ok=True)

# PARAMETERS
IMG_SIZE = (300, 300)
BATCH_SIZE = 32
EPOCHS = 30

# CENTER CROP FUNCTION
def center_crop_and_preprocess(img):
    """Crop to center square, resize, then preprocess for EfficientNetV2."""
    h, w, _ = img.shape
    min_side = min(h, w)
    top = (h - min_side) // 2
    left = (w - min_side) // 2
    img = img[top:top + min_side, left:left + min_side]
    img = tf.image.resize(img, IMG_SIZE)
    img = preprocess_input(img)
    return img

# DATA AUGMENTATION
train_datagen = ImageDataGenerator(
    preprocessing_function=center_crop_and_preprocess,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    shear_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest",
)

val_datagen = ImageDataGenerator(preprocessing_function=center_crop_and_preprocess)
test_datagen = ImageDataGenerator(preprocessing_function=center_crop_and_preprocess)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=True
)
val_gen = val_datagen.flow_from_directory(
    VAL_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)
test_gen = test_datagen.flow_from_directory(
    TEST_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

# CLASS WEIGHTS → for Focal Loss alpha
labels = train_gen.classes

class_counts = np.bincount(labels)
print("Counts per class:", class_counts)

# inverse-sqrt weighting
alpha_array = 1.0 / np.sqrt(class_counts)
alpha_array = alpha_array / np.sum(alpha_array)
print("New alpha_array:", alpha_array)

# LOSS FUNCTION
loss_fn = CategoricalFocalCrossentropy(
    gamma=1.5,
    alpha=alpha_array,
    label_smoothing=0.05,
    from_logits=False
)

# CALLBACKS
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(LOCAL_CHECKPOINT_PATH, "ckpt-{epoch:02d}.keras"),
    save_weights_only=False,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)
early_stop = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=4, min_lr=1e-6, verbose=1)


model = tf.keras.models.load_model('/content/drive/MyDrive/skin_cancer/models/efficientnetv2_checkpoints/ckpt-21.keras', compile=False)
# Re-compile with smaller LR
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-5),
    loss=loss_fn,
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), AUC(name="auc")]
)

history_finetune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    initial_epoch=21,
    callbacks=[checkpoint_callback, early_stop, reduce_lr],
    verbose=1
)

# SAVE FINAL MODEL
model.save(FINAL_MODEL_PATH, save_format="tf")
print(f"✅ Training complete! Full model saved at {FINAL_MODEL_PATH}")

Found 27205 images belonging to 7 classes.
Found 235 images belonging to 7 classes.
Found 1470 images belonging to 7 classes.
Counts per class: [3000 3500 4000 3000 4000 6705 3000]
New alpha_array: [0.15805851 0.14633375 0.13688269 0.15805851 0.13688269 0.10572535
 0.15805851]


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 22/30
851/851 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9402 - auc: 0.9966 - loss: 0.0639 - precision: 0.9642 - recall: 0.9001
Epoch 22: val_loss improved from inf to 0.09334, saving model to /content/drive/MyDrive/skin_cancer/models/efficientnetv2_checkpoints/ckpt-22.keras
851/851 ━━━━━━━━━━━━━━━━━━━━ 1130s 1s/step - accuracy: 0.9402 - auc: 0.9966 - loss: 0.0639 - precision: 0.9642 - recall: 0.9001 - val_accuracy: 0.8383 - val_auc: 0.9674 - val_loss: 0.0933 - val_precision: 0.8732 - val_recall: 0.7915 - learning_rate: 2.0000e-05
Epoch 23/30
851/851 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9471 - auc: 0.9972 - loss: 0.0611 - precision: 0.9665 - recall: 0.9189
Epoch 23: val_loss did not improve from 0.09334
851/851 ━━━━━━━━━━━━━━━━━━━━ 901s 1s/step - accuracy: 0.9471 - auc: 0.9972 - loss: 0.0611 - precision: 0.9665 - recall: 0.9189 - val_accuracy: 0.8255 - val_auc: 0.9624 - val_loss: 0.0962 - val_precision: 0.8472 - val_recall: 0.7787 - learning_rate: 2.0000e-05
Epoch 24/

KeyboardInterrupt: 

**evoluation after restart run time**

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Paths
CHECKPOINT_PATH = "/content/drive/MyDrive/skin_cancer/models/efficientnetv2_checkpoints/ckpt-22.keras"
TEST_DIR = "/content/final_data/test/sorted"
IMG_SIZE = (300, 300)
BATCH_SIZE = 32

# Preprocessing and Generators
def center_crop_and_preprocess(img):
    h, w, _ = img.shape
    min_side = min(h, w)
    top = (h - min_side) // 2
    left = (w - min_side) // 2
    img = img[top:top + min_side, left:left + min_side]
    img = tf.image.resize(img, IMG_SIZE)
    img = preprocess_input(img)
    return img

test_datagen = ImageDataGenerator(preprocessing_function=center_crop_and_preprocess)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Load Model & Evaluate
model = tf.keras.models.load_model(CHECKPOINT_PATH, compile=False)

steps = int(np.ceil(test_gen.samples / test_gen.batch_size))
preds = model.predict(test_gen, steps=steps, verbose=1)

y_true = test_gen.classes
y_pred = np.argmax(preds, axis=1)
target_names = list(test_gen.class_indices.keys())

print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
print("Macro F1:", f1_score(y_true, y_pred, average='macro'))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

Found 1470 images belonging to 7 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


46/46 ━━━━━━━━━━━━━━━━━━━━ 45s 699ms/step
              precision    recall  f1-score   support

       AKIEC     0.6154    0.5333    0.5714        30
         BCC     0.7125    0.6129    0.6590        93
         BKL     0.6527    0.7189    0.6842       217
          DF     0.8636    0.7600    0.8085        25
         MEL     0.5446    0.6433    0.5898       171
          NV     0.9097    0.8757    0.8924       909
        VASC     0.7692    0.8000    0.7843        25

    accuracy                         0.7986      1470
   macro avg     0.7240    0.7063    0.7128      1470
weighted avg     0.8076    0.7986    0.8019      1470

Macro F1: 0.7128017161655302
Confusion Matrix:
 [[ 16   3   7   0   2   2   0]
 [  4  57   8   1  13   8   2]
 [  5   8 156   0  26  22   0]
 [  0   1   2  19   0   3   0]
 [  0   1  16   0 110  42   2]
 [  1   8  50   2  50 796   2]
 [  0   2   0   0   1   2  20]]
